In [17]:
import pyxdf
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
%matplotlib qt

In [18]:
#Reading files
def read_file(subject_nr, session_nr, run_nr):
    dir = 'C:/Users/annej/OneDrive/Documents/Data collection'
    f_name = f'/sub-{subject_nr}/ses-{session_nr}/eeg/sub-{subject_nr}_ses-{session_nr}_task-Default_run-{run_nr}_eeg'
    f_format = '.xdf'
    fname = dir + f_name + f_format
    streams, header = pyxdf.load_xdf(fname)
    return streams, header

In [19]:
# Reading EEG
def configure_eeg_data(index, streams, header):   
    eeg_data = streams[index]['time_series'].T
    eeg_data *= 1e-6 # uV -> V
    time_stamps_eeg = streams[index]["time_stamps"] - streams[index]['time_stamps'][0]
    sfreq_eeg = float(streams[index]['info']['nominal_srate'][0])

    time_stamps_eeg = time_stamps_eeg.reshape((1,time_stamps_eeg.shape[0]))
    time_stamps_eeg = np.repeat(time_stamps_eeg, 8, axis=0)

    return eeg_data, time_stamps_eeg, sfreq_eeg


In [20]:
# Reading Markers
#'0' marker is standard and '1' marker is oddball/target
def configure_markers(index, streams, header):
    markers = streams[index]["time_series"]
    time_stamps = streams[index]["time_stamps"] - streams[index]["time_stamps"][0]

    return markers, time_stamps

In [21]:
#Reading pcg 
def configure_pcg_data(index, streams, header):
    pcg_data = streams[index]["time_series"].T
    sfreq_pcg = float(streams[index]["info"]["nominal_srate"][0])

    if len(streams[index]["time_stamps"]) == 0:
        print('ERROR: Found no time stamps')
        time_stamps_pcg = []
    else:
        time_stamps_pcg = streams[index]["time_stamps"] - streams[index]["time_stamps"][1]
        #plt.plot(pcg_data.T.ravel()/np.max(abs(pcg_data)))    
    
    return pcg_data, time_stamps_pcg, sfreq_pcg

In [22]:
def main(subject_nr, session_nr, run_nr):
    streams, header = read_file(subject_nr, session_nr, run_nr)

    #Directing each stream into correct function
    for i in range(len(streams)):
        print(i)
        if streams[i]['info']['type'] == ['Audio']:
            print('Found pcg data')
            pcg_data, time_stamps_pcg, sfreq_pcg = configure_pcg_data(i, streams, header)
        elif streams[i]['info']['type'] == ['ExG']:
            print('Found eeg data')
            eeg_data, time_stamps_eeg, sfreq_eeg = configure_eeg_data(i, streams, header)
        elif streams[i]['info']['type'] == ['Markers']:
            print('Found markers')
            markers, time_stamps_markers = configure_markers(i, streams, header)
        else:
            print('Found nothing')

    # Creating the raw data
    info = mne.create_info(8, sfreq_eeg, ch_types= "eeg", verbose=None)
    print(info)
    raw = mne.io.RawArray(eeg_data, info)

    # Renaming the channels and dropping the non-used channels
    mapping = {'0':'F4','1':'Fp2','2':'C3',
            '3':'FC6','4':'O1','5':'Oz','6':'FT9', 
            '7':'T8'}
    mne.rename_channels(raw.info, mapping)

    title = f"sub-{subject_nr}_ses-{session_nr}_run-{run_nr}"
    data_frame = raw.to_data_frame(scalings=1)
    data = data_frame.to_numpy()
    data = np.transpose(data)
    data_dict = {
        "raw_eeg_data" : data[1:, :]  #First column of dataFrames is not data
        }
    scipy.io.savemat(f'Data/Raw_eeg/{title}.mat', data_dict)


In [23]:
#raw.plot(scalings=dict(eeg=50e-6),duration=10)

#raw.plot(duration=60, title='test', n_channels=8,show_scrollbars=False, highpass = 1, lowpass = 40)

In [30]:
n_sub = 27
subject_nr = ['P001', 'P002', 'P003', 'P004', 'P005', 'P006', 'P007', 'P008', 'P009', 'P010',
              'P011', 'P012', 'P013', 'P014', 'P015', 'P016', 'P017', 'P018', 'P019', 'P020',
              'P021', 'P022', 'P023', 'P024', 'P025', 'P026', 'P027']

subject_nr = ['P024', 'P025', 'P026', 'P027']

session_nr = ['S001', 'S002']

run_nr =     ['001', '002']


for i in range(len(subject_nr)):
    print(f'Subject: {subject_nr[i]}')
    for j in range(len(session_nr)):
        print(f'Session: {session_nr[j]}')
        for k in range(len(run_nr)):
            print(f'Run: {run_nr[k]}')
            main(subject_nr = subject_nr[i], session_nr = session_nr[j], run_nr = run_nr[k])

Subject: P024
Session: S001
Run: 001
0
Found eeg data
1
Found pcg data
<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>
Creating RawArray with float64 data, n_channels=8, n_times=75184
    Range : 0 ... 75183 =      0.000 ...   300.732 secs
Ready.
Run: 002
0
Found eeg data
1
Found markers
2
Found pcg data
<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>
Creating RawArray with float64 data, n_channels=8, n_times=75296
    Range : 0 ... 75295 =      0.000 ...   301.180 secs
Ready.
Session: S002
Run: 001
0
Found eeg data
1
Found pcg data
<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpas